In [1]:
%reload_ext autoreload
%autoreload 2

import pydicom
import pynetdicom
from pynetdicom import AE
from pydicom import uid
import funcs
import os
import sys
sys.path.append('.')

%reload_ext funcs

## Send a C-echo request to a verification SCP

Verification SOP Class has a UID of:  1.2.840.10008.1.1

we can use the UID str directly when adding the requested presentation context

# <span style='color:green'> Supported DIMSE Services </span>
## <span style='color:orange'> SCU Services </span>
When the AE is acting as an SCU and an association has been established with a peer SCP, the following DIMSE-C and -N services are available:


| DIMSE service	| Association method |
|---	|---|
| C-ECHO	| Association.send_c_echo() |
| C-FIND	| Association.send_c_find(dataset, query_model) |
| C-GET	| Association.send_c_get(dataset, query_model) |
| C-MOVE	| Association.send_c_move(dataset, move_aet, query_model) |
| C-STORE	| Association.send_c_store(dataset) |
| N-ACTION	| Association.send_n_action(dataset, action_type, class_uid, instance_uid) |
| N-CREATE	| Association.send_n_create(dataset, class_uid, instance_uid) |
| N-DELETE	| Association.send_n_delete(class_uid, instance_uid) |
| N-EVENT-REPORT	| Association.send_n_event_report(dataset, event_type, class_uid, instance_uid) |
| N-GET	| Association.send_n_get(identifier_list, class_uid, instance_uid) |
| N-SET	| Association.send_n_set(dataset, class_uid, instance_uid) |

In [46]:
addr = "www.dicomserver.co.uk"
port = 104 # 11112
ae_title="AET"
uid_verification_sop_class = '1.2.840.10008.1.1'


# Associate with a peer AE at IP
ae = AE(ae_title=ae_title)

ae.add_requested_context(uid_verification_sop_class)

associate = ae.associate(addr=addr, port=port)
print('associate is' + ('' if associate.is_established else 'not') + ' established')

# Example: Send a DIMSE C-ECHO request to the peer AE </span>
status = associate.send_c_echo()
print(status)

# associate.release()

associate is established
(0000, 0900) Status                              US: 0


# `send_c_find`

```python
AE().associate.send_c_get(dataset: Dataset, query_model: Union[str, UID], msg_id: int = 1, priority: int = 2)→ Iterator[Tuple[Dataset, Optional[Dataset]]][source]¶

```

Parameters
- `dataset` (`pydicom.dataset.Dataset`) – The `C-GET` request’s Identifier dataset. The exact requirements for the Identifier are Service Class specific (see the DICOM Standard, Part 4).

- `query_model` (`pydicom.uid.UID` or `str`) – The value to use for the `C-GET` request’s `(0000,0002)` Affected SOP Class UID parameter, which usually corresponds to the Information Model that is to be used.

- `msg_id` (`int`, `optional`) – The `C-GET` request’s Message ID, must be between `0` and `65535`, inclusive, (**`default 1`**).

- `priority` (`int`, `optional`) – The `C-GET` request’s Priority parameter, must be `0` (`Medium`), `1` (`High`) or `2` (`Low`) (**`default 2`**)




```bash
python -m pynetdicom findscu www.dicomserver.co.uk 104 -k QueryRetrieveLevel=STUDY

```

In [19]:
from pynetdicom import AE, sop_class

connect_To_PACS = funcs.Connect_To_PACS()

connect_To_PACS.send_c_find(show_results=False, queryRetrieveLevel='STUDY', requestedContext=sop_class.PatientRootQueryRetrieveInformationModelFind)

In [31]:
connect_To_PACS.list_sample_info[10][0]

(0000, 0900) Status                              US: 65280

# `send_c_get`

In [21]:
import pydicom
from pynetdicom import AE, evt, build_role, debug_logger, sop_class
from pydicom import uid


addr = "www.dicomserver.co.uk"
port = 104 # 11112
ae_title="AET"


debug_logger()

# Implement the handler for evt.EVT_C_STORE
def handle_store(event):
    """Handle a C-STORE request event."""
    ds = event.dataset
    ds.file_meta = event.file_meta

    # Save the dataset using the SOP Instance UID as the filename
    ds.save_as(ds.SOPInstanceUID + '.dcm', write_like_original=False)

    # Return a 'Success' status
    return 0x0000

handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE()

# Add the requested presentation contexts (QR SCU)
ae.add_requested_context(sop_class.PatientRootQueryRetrieveInformationModelGet)

context = sop_class.CTImageStorage
# Add the requested presentation context (Storage SCP)
ae.add_requested_context(context)

# Create an SCP/SCU Role Selection Negotiation item for CT Image Storage
role = build_role(context, scp_role=True)

# Create our Identifier (query) dataset
# We need to supply a Unique Key Attribute for each level above the
#   Query/Retrieve level
ds = pydicom.dataset.Dataset()
ds.QueryRetrieveLevel = 'PATIENT'

# Unique key for PATIENT level
ds.PatientID = 'NOID'

# Unique key for STUDY level
# ds.StudyInstanceUID = '1.2.3'

# Unique key for SERIES level
# ds.SeriesInstanceUID = '2'

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate(addr=addr, port=port, ext_neg=[role], evt_handlers=handlers)

if assoc.is_established:
    # Use the C-GET service to send the identifier
    responses = assoc.send_c_get(ds, sop_class.PatientRootQueryRetrieveInformationModelGet)
    
    for (status, identifier) in responses:
        if status:
            print('C-GET query status: 0x{0:04x}'.format(status.Status))
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    # assoc.release()
else:
    print('Association rejected, aborted or never connected')

I: Requesting Association
D: Request Parameters:
D: ======================= OUTGOING A-ASSOCIATE-RQ PDU ========================
D: Our Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.0.2
D: Our Implementation Version Name:   PYNETDICOM_202
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    PYNETDICOM
D: Called Application Name:     ANY-SCP
D: Our Max PDU Receive Size:    16382
D: Presentation Contexts:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =Patient Root Query/Retrieve Information Model - GET
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D:   Context ID:        3 (Proposed)
D:     Abstract Syntax: =CT Image Storage
D:     Proposed SCP/SCU Role: SCP
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Lit

C-GET query status: 0xff00


I: Received Store Request
D: ========================== INCOMING DIMSE MESSAGE ==========================
D: Message Type                  : C-STORE RQ
D: Presentation Context ID       : 3
D: Message ID                    : 17308
D: Affected SOP Class UID        : CT Image Storage
D: Affected SOP Instance UID     : 1.3.6.1.4.1.5962.99.1.1761388472.1291962045.1616669124536.2595.0
D: Data Set                      : Present
D: Priority                      : Medium
D: ============================ END DIMSE MESSAGE =============================
D: pydicom.read_dataset() TransferSyntax="Little Endian Explicit"
D: pydicom.read_dataset() TransferSyntax="Little Endian Implicit"
D: ========================== INCOMING DIMSE MESSAGE ==========================
D: Message Type                  : C-GET RSP
D: Presentation Context ID       : 1
D: Message ID Being Responded To : 1
D: Affected SOP Class UID        : Patient Root Query/Retrieve Information Model - GET
D: Remaining Sub-operations      : 

C-GET query status: 0xff00


I: Received Store Request
D: ========================== INCOMING DIMSE MESSAGE ==========================
D: Message Type                  : C-STORE RQ
D: Presentation Context ID       : 3
D: Message ID                    : 17309
D: Affected SOP Class UID        : CT Image Storage
D: Affected SOP Instance UID     : 1.3.6.1.4.1.5962.99.1.1761388472.1291962045.1616669124536.2597.0
D: Data Set                      : Present
D: Priority                      : Medium
D: ============================ END DIMSE MESSAGE =============================
D: pydicom.read_dataset() TransferSyntax="Little Endian Explicit"
D: pydicom.read_dataset() TransferSyntax="Little Endian Implicit"
D: ========================== INCOMING DIMSE MESSAGE ==========================
D: Message Type                  : C-GET RSP
D: Presentation Context ID       : 1
D: Message ID Being Responded To : 1
D: Affected SOP Class UID        : Patient Root Query/Retrieve Information Model - GET
D: Remaining Sub-operations      : 

...

C-GET query status: 0xff00


I: Received Store Request
D: ========================== INCOMING DIMSE MESSAGE ==========================
D: Message Type                  : C-STORE RQ
D: Presentation Context ID       : 3
D: Message ID                    : 17325
D: Affected SOP Class UID        : CT Image Storage
D: Affected SOP Instance UID     : 1.3.6.1.4.1.5962.99.1.1761388472.1291962045.1616669124536.2615.0
D: Data Set                      : Present
D: Priority                      : Medium
D: ============================ END DIMSE MESSAGE =============================
D: pydicom.read_dataset() TransferSyntax="Little Endian Explicit"
D: pydicom.read_dataset() TransferSyntax="Little Endian Implicit"
D: ========================== INCOMING DIMSE MESSAGE ==========================
D: Message Type                  : C-GET RSP
D: Presentation Context ID       : 1
D: Message ID Being Responded To : 1
D: Affected SOP Class UID        : Patient Root Query/Retrieve Information Model - GET
D: Remaining Sub-operations      : 

C-GET query status: 0xff00


I: Received Store Request
D: ========================== INCOMING DIMSE MESSAGE ==========================
D: Message Type                  : C-STORE RQ
D: Presentation Context ID       : 3
D: Message ID                    : 17326
D: Affected SOP Class UID        : CT Image Storage
D: Affected SOP Instance UID     : 1.3.6.1.4.1.5962.99.1.1761388472.1291962045.1616669124536.2616.0
D: Data Set                      : Present
D: Priority                      : Medium
D: ============================ END DIMSE MESSAGE =============================


KeyboardInterrupt: 

# Other Stuff

In [49]:
from dataclasses import dataclass

# from pynetdicom.sop_class import VerificationSOPClass
from pynetdicom.sop_class import VerificationServiceClass, Verification


@dataclass
class Modality:
    addr: str
    port: int
    ae_title: str

class Association:
    def __init__(self, modality, context):
        self.modality = modality
        self.context = context

    def __enter__(self):
        ae = AE()
        ae.add_requested_context(self.context)
        self._association = ae.associate(**vars(self.modality))
        return self._association

    def __exit__(self, *args):
        self._association.release()
        self._association = None
        
        
modality = Modality('www.dicomserver.co.uk', 104, 'AET')


if __name__ == '__main__':
    modality = Modality('www.dicomserver.co.uk', 104, 'AET')

    with Association(modality, Verification) as assoc:
        resp = assoc.send_c_echo()
        # assoc.req
        print(f'Modality responded with status: {resp.Status}')
        
        # request the list of studies
        resp = assoc.send_c_find()
        # for (status, dataset) in resp:
        #     print(f'Modality responded with status: {status}')
        #     if dataset:
        #         print(dataset)


Requesting Association

Association Accepted

Sending Echo Request: MsgID 1

Received Echo Response (Status: 0x0000 - Success)

Modality responded with status: 0


Releasing Association

TypeError: Association.send_c_find() missing 2 required positional arguments: 'dataset' and 'query_model'